In [4]:
import cv2
import numpy as np
import os
from scipy.interpolate import UnivariateSpline

In [5]:
def load_frames_from_directory(frames_path):
    """Load frame file paths from directory in sorted order"""
    frame_files = []
    for filename in sorted(os.listdir(frames_path)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            frame_files.append(os.path.join(frames_path, filename))
    return frame_files

def detect_and_match_features_akaze(img1, img2):
    # AKAZE is robust and fast
    akaze = cv2.AKAZE_create()
    kp1, des1 = akaze.detectAndCompute(img1, None)
    kp2, des2 = akaze.detectAndCompute(img2, None)
    if des1 is None or des2 is None or len(kp1) < 4 or len(kp2) < 4:
        return np.array([]), np.array([])
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    if len(matches) < 4:
        return np.array([]), np.array([])
    good_matches = matches[:min(50, len(matches))]
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches])
    return pts1, pts2

def filter_motion_outliers(pts1, pts2):
    # Remove matches with unusually large motion (likely moving objects)
    motion_vectors = pts2 - pts1
    motion_magnitudes = np.linalg.norm(motion_vectors, axis=1)
    median_motion = np.median(motion_magnitudes)
    mad = np.median(np.abs(motion_magnitudes - median_motion))
    threshold = median_motion + 2 * mad
    static_mask = motion_magnitudes < threshold
    return pts1[static_mask], pts2[static_mask]

def estimate_affine_transform(pts1, pts2):
    if len(pts1) >= 3 and len(pts2) >= 3:
        M, inliers = cv2.estimateAffinePartial2D(pts1, pts2, method=cv2.RANSAC)
        if M is not None:
            # Convert to 3x3 homogeneous
            H = np.vstack([M, [0, 0, 1]])
            return H
    return np.eye(3)

def smooth_trajectory(trajectory, smoothing_factor=5):
    trajectory = np.array(trajectory)
    smoothed = []
    for i in range(trajectory.shape[1]):
        x = np.arange(trajectory.shape[0])
        y = trajectory[:, i]
        spl = UnivariateSpline(x, y, s=smoothing_factor)
        smoothed.append(spl(x))
    return np.array(smoothed).T

def streaming_video_stabilization_akaze_from_frames(frames_path, output_path, fps=29.97):
    """
    AKAZE stabilization using pre-extracted frames from directory
    """
    # Load frame file paths
    frame_files = load_frames_from_directory(frames_path)
    total_frames = len(frame_files)
    
    if total_frames == 0:
        print("Error: No frames found in directory")
        return
    
    print(f"Found {total_frames} frames in directory")
    
    # Read first frame to get dimensions
    first_frame = cv2.imread(frame_files[0])
    if first_frame is None:
        print("Error: Could not read first frame")
        return
    
    h, w = first_frame.shape[:2]
    
    # Setup output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))
    
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    transforms = []
    trajectory = [np.eye(3)]
    
    print("Phase 1: Computing transforms (using AKAZE+Affine from frames)...")
    
    # Process frames from directory
    for i in range(1, total_frames):
        curr_frame = cv2.imread(frame_files[i])
        if curr_frame is None:
            print(f"Warning: Could not read frame {i}")
            transforms.append(np.eye(3))
            trajectory.append(trajectory[-1])
            continue
        
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        pts1, pts2 = detect_and_match_features_akaze(prev_gray, curr_gray)
        
        if len(pts1) > 0:
            pts1, pts2 = filter_motion_outliers(pts1, pts2)
        
        H = estimate_affine_transform(pts1, pts2)
        transforms.append(H)
        trajectory.append(trajectory[-1] @ H)
        prev_gray = curr_gray
        
        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1}/{total_frames} frames")
    
    print("Phase 2: Smoothing trajectory...")
    
    # Extract parameters and smooth
    params = []
    for H in trajectory:
        dx = H[0, 2]
        dy = H[1, 2]
        da = np.arctan2(H[1, 0], H[0, 0])
        params.append([dx, dy, da])
    
    smoothed_params = smooth_trajectory(params, smoothing_factor=10)
    
    # Calculate smoothed transforms
    new_transforms = []
    for i in range(len(smoothed_params)):
        dx, dy, da = smoothed_params[i]
        cos_a = np.cos(da)
        sin_a = np.sin(da)
        new_H = np.array([[cos_a, -sin_a, dx],
                          [sin_a,  cos_a, dy],
                          [0,      0,     1]], dtype=np.float32)
        new_transforms.append(new_H)
    
    print("Phase 3: Applying stabilization and writing video...")
    
    # Apply stabilization and write video
    for i in range(total_frames):
        frame = cv2.imread(frame_files[i])
        if frame is None:
            continue
        
        if i < len(new_transforms):
            H = new_transforms[i]
            if H is not None and not np.isnan(H).any() and not np.isinf(H).any():
                try:
                    frame_stabilized = cv2.warpPerspective(frame, H, (w, h), borderMode=cv2.BORDER_REFLECT)
                except:
                    frame_stabilized = frame
            else:
                frame_stabilized = frame
        else:
            frame_stabilized = frame
        
        out.write(frame_stabilized)
        
        if (i + 1) % 100 == 0:
            print(f"Stabilized and wrote {i + 1}/{total_frames} frames")
    
    out.release()
    print(f"Stabilized video saved at: {output_path}")


In [6]:
frames_path = r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\unzipped_video"
output_path = r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\akaze_affine_from_frames_output.mp4"
streaming_video_stabilization_akaze_from_frames(frames_path, output_path, fps=29.97)

Found 7169 frames in directory
Phase 1: Computing transforms (using AKAZE+Affine from frames)...
Processed 100/7169 frames
Processed 200/7169 frames
Processed 300/7169 frames
Processed 400/7169 frames
Processed 500/7169 frames
Processed 600/7169 frames
Processed 700/7169 frames
Processed 800/7169 frames
Processed 900/7169 frames
Processed 1000/7169 frames
Processed 1100/7169 frames
Processed 1200/7169 frames
Processed 1300/7169 frames
Processed 1400/7169 frames
Processed 1500/7169 frames
Processed 1600/7169 frames
Processed 1700/7169 frames
Processed 1800/7169 frames
Processed 1900/7169 frames
Processed 2000/7169 frames
Processed 2100/7169 frames
Processed 2200/7169 frames
Processed 2300/7169 frames
Processed 2400/7169 frames
Processed 2500/7169 frames
Processed 2600/7169 frames
Processed 2700/7169 frames
Processed 2800/7169 frames
Processed 2900/7169 frames
Processed 3000/7169 frames
Processed 3100/7169 frames
Processed 3200/7169 frames
Processed 3300/7169 frames
Processed 3400/7169 f